In [1]:
!pip install scikit-learn

You should consider upgrading via the 'C:\Users\micha\anaconda3\envs\AICH\python.exe -m pip install --upgrade pip' command.


In [22]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler

In [5]:
train = pd.read_parquet('../data/processed/train_series_lightweight.parquet')

In [7]:
train.head()

,series_id,step,timestamp,anglez,enmo,awake,anglez_median,anglez_mean,anglez_min,anglez_max,anglez_diff,enmo_median,enmo_mean,enmo_min,enmo_max,enmo_diff,anglezxenmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217,1,2.6367,7.573975,2.4129,54.8498,41.462601,0.02165,0.02233,0.0166,0.0395,-0.0047,0.057216
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.6368,0.0215,1,2.6367,7.573975,2.4129,54.8498,41.462601,0.02165,0.02233,0.0166,0.0395,-0.0047,0.056691
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.6370,0.0216,1,2.6367,7.573975,2.4129,54.8498,41.462601,0.02165,0.02233,0.0166,0.0395,-0.0047,0.056959
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.6368,0.0213,1,2.6367,7.573975,2.4129,54.8498,41.462601,0.02165,0.02233,0.0166,0.0395,-0.0047,0.056164
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.6368,0.0215,1,2.6367,7.573975,2.4129,54.8498,41.462601,0.02165,0.02233,0.0166,0.0395,-0.0047,0.056691


## Hyperparams

In [8]:
train.columns

Index(['series_id', 'step', 'timestamp', 'anglez', 'enmo', 'awake',
       'anglez_median', 'anglez_mean', 'anglez_min', 'anglez_max',
       'anglez_diff', 'enmo_median', 'enmo_mean', 'enmo_min', 'enmo_max',
       'enmo_diff', 'anglezxenmo'],
      dtype='object')

In [9]:
RANDOM_STATE = 42
features = ['anglez', 'enmo', 'anglez_median', 'anglez_mean', 'anglez_min', 'anglez_max',
       'anglez_diff', 'enmo_median', 'enmo_mean', 'enmo_min', 'enmo_max',
       'enmo_diff', 'anglezxenmo']

## Modeling

In [38]:
transformer = RobustScaler().fit(train[features])

X, y = transformer.transform(train[features]), train['awake']

In [24]:
lr = LinearRegression().fit(X, y)

## Prediction

In [29]:
series = pd.read_parquet('../data/processed/train_series.parquet', filters=[('series_id','=','1087d7b0ff2e')])

Select rows between step 7000 and 7500, as in between the state changes from awake to asleep

In [43]:
test_X = series[(series.step < 7500) & (series.step > 7000)][features]
y_true = series[(series.step < 7500) & (series.step > 7000)]['awake']

In [39]:
test_X_transformed = transformer.transform(test_X)

In [44]:
y_pred = lr.predict(test_X_transformed)

## Scores

In [55]:
from sklearn.metrics import mean_absolute_percentage_error

In [56]:
mean_absolute_percentage_error(y_true, y_pred)

984689100000000.0

## Feature importance

In [63]:
for i, value in enumerate(lr.coef_):
    print(f'Feature {features[i]}: {value}')

Feature anglez: 0.007205110974609852
Feature enmo: 0.0066049108281731606
Feature anglez_median: -0.010910134762525558
Feature anglez_mean: 8.907479059416801e-05
Feature anglez_min: -0.12739421427249908
Feature anglez_max: 0.11094117164611816
Feature anglez_diff: 0.00045819810475222766
Feature enmo_median: 0.023408059030771255
Feature enmo_mean: -0.004690743051469326
Feature enmo_min: 0.04929041117429733
Feature enmo_max: 0.00047453708248212934
Feature enmo_diff: -0.0018653966253623366
Feature anglezxenmo: -0.001359041198156774
